# Experiments

In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [2]:
auth = {
    "uri": "bolt://localhost:7687",
    "user": "neo4j",
    "password": "12345678"
}

In [29]:
def read_from_db(query, auth, db):
    '''
    This function allows us to query the active DB using the specified
    authentication information, and return the results as a DataFrame
    '''
    driver = GraphDatabase.driver(auth['uri'], auth=(auth['user'], auth['password']))
    records, summary, keys = driver.execute_query(query, database_=db)
    driver.close()

    print(f'The query returned {len(records)} records in {summary.result_available_after} ms.')
    df = pd.DataFrame([dict(record) for record in records])
    return df

In [ ]:
query = 'MATCH (n:Customer) RETURN n.customer_id, n.x_customer_id, n.y_customer_id'
# query = 'MATCH (n) DETACH DELETE n'

df = read_from_db(query, auth)
df.head()

## Query (A)

**Query:** For each customer check that the spending frequency and the spending amounts of the last month are under the usual spending frequency and the spending amounts for the same period.

**Assumptions:**

- For this query, we should focus solely on transactions from the past month. As the generated dataset may span along various timeframes (past, present, and future), we can't simply use `today()`'s date as a reference point to capture transactions from the preceding month. Instead, we rely on the latest recorded transaction in the entire database, and we include transactions up to 31 days before that date.
- For a given customer $c$ and customer's set of transactions $t \in \{transactions\_last\_month\}$, we assume that:

  $spending\_frequency = count(t)$

  $spending\_amount = \sum_{t} t.tx\_amount$
  
- In order to compare such values with the *usual* spending frequency and amount of all customers $c \in C$ in the same period, we will compute $avg\_frequency$ and $avg\_amount$ as reference values to perform the comparison:

  $avg\_frequency = \frac{1}{|C|} \sum_{c \in C} c.spending\_frequency$

  $avg\_amount = \frac{1}{|C|} \sum_{c \in C} c.spending\_amount$

- In the end, for each $c \in C$, we will report whether or not $(c.spending\_frequency < avg\_frequency)$ and $(c.spending\_amount < avg\_amount)$.

In [4]:
query_A = '''
    MATCH ()-[t:TRANSACTION]->()
    WITH max(datetime(t.tx_datetime)) AS last_date
    
    MATCH (c:Customer)-[t:TRANSACTION]->(:Terminal)
    WHERE duration.inDays(datetime(t.tx_datetime), last_date).days < 31
    WITH
        c.customer_id AS customer_id,
        round(sum(t.tx_amount), 3) AS customer_transaction_amount,
        COUNT(t) AS customer_transaction_frequency
    WITH
        collect({
            id: customer_id,
            tx_amount: customer_transaction_amount,
            tx_freq: customer_transaction_frequency
        }) AS customer_details,
        round(avg(customer_transaction_amount), 3) AS average_transaction_amount,
        avg(customer_transaction_frequency) AS average_transaction_frequency
    UNWIND customer_details as cd
    RETURN
        cd.id AS customer_id,
        cd.tx_amount AS customer_transaction_amount,
        average_transaction_amount,
        cd.tx_freq AS customer_transaction_frequency,
        average_transaction_frequency,
        CASE
            WHEN cd.tx_amount < average_transaction_amount
            THEN 'Lower than average' ELSE 'Higher than average'
            END AS amount_comparison,
        CASE
            WHEN cd.tx_freq < average_transaction_frequency
            THEN 'Lower than average' ELSE 'Higher than average'
            END AS frequency_comparison
'''

In [33]:
df_A = read_from_db(query_A, auth, 'TG50')
df_A.head()

The query returned 0 records in 2 ms.


""


## Query (B)

**Query:** For each terminal identify the possible fraudulent transactions. The fraudulent transactions are those whose import is higher than 20% of the maximum import of the transactions executed on the same terminal in the last month.

**Assumptions:**
- Similar to query (A), we obtain the latest transaction recorded in DB according to the previously mentioned reasons.
- To implement the query, we need to establish a `fraud_threshold` for each terminal. Initially, the description suggests setting the threshold as $0.2 \times \text{max}(t)$, resulting in numerous transactions $t \in \{terminal\_transactions\}$ being flagged as fraudulent. Instead, we propose setting the threshold as $0.8 \times \text{max}(t)$ to obtain only transactions with abnormal amounts for each terminal in the last month.

In [6]:
query_B = '''
    MATCH ()-[t:TRANSACTION]->()
    WITH max(datetime(t.tx_datetime)) AS last_date

    MATCH (:Customer)-[t:TRANSACTION]->(terminal:Terminal)
    WHERE duration.inDays(datetime(t.tx_datetime), last_date).days < 31
    WITH terminal, round(0.8 * max(t.tx_amount), 3) AS fraud_threshold, collect(t) AS transactions
    UNWIND transactions AS transaction
    WITH terminal, transaction, fraud_threshold
    WHERE transaction.tx_amount > fraud_threshold
    RETURN
        terminal.terminal_id AS terminal_id,
        transaction.transaction_id AS transaction_id,
        transaction.customer_id AS customer_id,
        transaction.tx_amount AS transaction_amount,
        fraud_threshold
'''

In [7]:
df_B = read_from_db(query_B, auth)
df_B.head()

The query returned 5 records in 272 ms.


,terminal_id,transaction_id,customer_id,transaction_amount,fraud_threshold
0,0,26,1,1546.25,1237.0
1,0,20,1,1381.50,1237.0
2,3,44,1,7981.25,6385.0
3,4,40,4,1844.25,1475.4
4,4,43,4,1674.25,1475.4


## Query (C)

**Query:** Given a user $u$, determine the “co-customer-relationships $CC$ of degree $k$”. A user $u’$ is a co-customer of $u$ if you can determine a chain “$u_1-t_1-u_2-t_2-…t_{k-1}-u_k$“ such that $u_1=u$, $u_k=u’$, and for each $1 \leq i$ and $j \leq k$, $u_i<>u_j$, and $t_1,..t_{k-1}$ are the terminals on which a transaction has been executed. Therefore, $CC_k(u)=\{u’| \text{ a chain exists between } u \text{ and } u’ \text{ of degree } k\}$. Please, note that depending on the adopted model, the computation of $CC_k(u)$ could be quite complicated. Consider therefore at least the computation of $CC_3(u)$ (i.e. the co-costumer relationships of degree 3).

**Assumptions:**
- According to the query description, to identify $CC_3(u)$ on our graph database, we should look for subgraphs similar to the diagram drawn below, and identify such co-customers $c_1=u$, $c_3=u’$:
  
  ![CC3-diagram](images/CC3-diagram.png)

- To compute $CC_k(u)$, it has been mentioned that each customer should be traversed once only ($u_i<>u_j$). However, nothing has been mentioned for the traverse of the terminals, hence we assume that they can be traversed repeatedly.
- In most cases, there are several transactions present among each customer and the connected terminals (possibly hundreds or thousands). To make the query more efficient, since we are only interested in identifying $CC_3(u)$ relationships and not all the possible chains between $u$ and $u’$, we group our data by the start/end nodes of the chain and only report a single chain as a sample using `head(collect(...))`.

In [8]:
query_C = '''
    MATCH
        (c1:Customer)-[:TRANSACTION]->(t1:Terminal),
        (t1)<-[:TRANSACTION]-(c2:Customer),
        (c2)-[:TRANSACTION]->(t2:Terminal),
        (t2)<-[:TRANSACTION]-(c3:Customer)
    WHERE c1 <> c2 AND c1 <> c3 AND c2 <> c3
    WITH
        c1,
        c3,
        head(collect({
            c2: c2.customer_id,
            t1: t1.terminal_id,
            t2: t2.terminal_id})
        ) as sample_chain
    RETURN
        c1.customer_id AS customer,
        c3.customer_id AS co_customer,
            "(c" + c1.customer_id + ")->(t" + sample_chain.t1
            + ")<-(c" + sample_chain.c2 + ")->(t" + sample_chain.t2
            + ")<-(c" + c3.customer_id + ")"
        AS sample_chain
'''

In [9]:
df_C = read_from_db(query_C, auth)
df_C.head()

The query returned 12 records in 334 ms.


,customer,co_customer,sample_chain
0,0,1,(c0)->(t4)<-(c4)->(t0)<-(c1)
1,0,3,(c0)->(t4)<-(c4)->(t0)<-(c3)
2,0,4,(c0)->(t4)<-(c1)->(t0)<-(c4)
3,1,4,(c1)->(t0)<-(c0)->(t4)<-(c4)
4,1,3,(c1)->(t0)<-(c0)->(t0)<-(c3)


## Query (D)

**Query:** Extend the logical model that you have stored in the NOSQL database by introducing the
following information:
- Each transaction should be extended with:
    1. The period of the day {morning, afternoon, evening, night} in which the transaction has been executed.
    2. The kind of products that have been bought through the transaction {hightech, food, clothing, consumable, other}
    3. The feeling of security expressed by the user. This is an integer value between 1 and 5 expressed by the user when conclude the transaction. The values can be chosen randomly.
- Customers that make more than three transactions from the same terminal expressing a similar average feeling of security should be connected as “buying_friends”. Therefore also this kind of relationship should be explicitly stored in the NOSQL database and can be queried. Note, two average feelings of security are considered similar when their difference is lower than 1.

**Assumptions:**
- To obtain the period of the day, we utlize the transaction hours such that transactions within "06:00-12:00" are tagged "morning", "12:00-18:00" as "afternoon", "18:00-00:00" as "evening", and "00:00-06:00" as "night".
- For the kind of product purchased along the transaction, we assume that the products only belong to a single category among {"high-tech", "food", "clothing", "consumable", "other"}, which we randomly select and assign to each transaction. If we instead wanted to have a *list* of products for each transaction, we could create a list such that each entry (corresponding to a product) is randomly included or excluded. For example, `CASE WHEN rand()<0.5 THEN "high-tech" END`, and we repeat the same for all other products.
- Security feeling of the transaction is also drawn randomly and distributed as an integer number from 1 to 5.
- In a separate query, we attempt to add `BUYING_FRIEND` relationships. We find customers with +3 transactions on the same terminal, computing their average security feeling separately, and connect them if their feelings is similar.

  ![buying_friends](images/buying-friends-diagram.png)

In [18]:
query_D1 = '''
    MATCH (:Customer)-[t:TRANSACTION]->(:Terminal)
    SET
        t.period_of_day = 
            CASE
                WHEN datetime(t.tx_datetime).hour >= 6 AND datetime(t.tx_datetime).hour < 12 THEN 'morning'
                WHEN datetime(t.tx_datetime).hour >= 12 AND datetime(t.tx_datetime).hour < 18 THEN 'afternoon'
                WHEN datetime(t.tx_datetime).hour >= 18 AND datetime(t.tx_datetime).hour < 24 THEN 'evening'
                ELSE 'night'
            END,
        t.product = 
            ["high-tech", "food", "clothing", "consumable", "other"]
            [toInteger(round(rand() * 4))],
        t.security_feeling = toInteger(round(rand() * 4) + 1)

    RETURN
        t.transaction_id, t.customer_id, t.terminal_id, t.tx_datetime,
        t.period_of_day, t.product, t.security_feeling
    LIMIT 5
'''

In [19]:
df_D1 = read_from_db(query_D1, auth)
df_D1.head()

The query returned 5 records in 107 ms.


,t.transaction_id,t.customer_id,t.terminal_id,t.tx_datetime,t.period_of_day,t.product,t.security_feeling
0,39,0,0,2018-04-04T01:15:35.000000000,night,clothing,3
1,55,4,0,2018-04-05T18:40:40.000000000,evening,food,5
2,41,1,0,2018-04-04T05:47:33.000000000,night,consumable,1
3,54,4,0,2018-04-05T17:23:34.000000000,afternoon,other,5
4,48,1,0,2018-04-05T12:16:56.000000000,afternoon,food,4


In [20]:
query_D2 = '''
    MATCH (c1:Customer)-[t1:TRANSACTION]->(tm1:Terminal)
    WITH c1, tm1, COUNT(t1) AS n_transactions_c1, avg(t1.security_feeling) AS avg_security_t1
    WHERE n_transactions_c1 > 3
    
    MATCH (c2:Customer)-[t2:TRANSACTION]->(tm2:Terminal)
    WITH c2, tm2, COUNT(t2) AS n_transactions_c2, avg(t2.security_feeling) AS avg_security_t2, c1, tm1, avg_security_t1
    WHERE n_transactions_c2 > 3 AND tm1=tm2 AND c1<>c2
        AND ABS(avg_security_t2 - avg_security_t1) < 1
    
    MERGE (c1)-[:BUYING_FRIENDS]->(c2)
    
    RETURN
        c1.customer_id, c2.customer_id,
        tm2.terminal_id AS terminal,
        avg_security_t1, avg_security_t2
'''

In [21]:
df_D2 = read_from_db(query_D2, auth)
df_D2.head()

The query returned 8 records in 23 ms.


,c1.customer_id,c2.customer_id,terminal,avg_security_t1,avg_security_t2
0,0,4,0,2.500000,3.250000
1,0,1,0,2.500000,3.166667
2,4,0,0,3.250000,2.500000
3,4,1,0,3.250000,3.166667
4,1,0,0,3.166667,2.500000


![buying_friends](images/buying-friends.png)

## Query (E)

**Query:** 

**Assumptions:**

In [25]:
query_E = '''
    MATCH (:Customer)-[t:TRANSACTION]-(:Terminal)
    RETURN
        t.period_of_day AS period_of_day,
        count(t) AS transaction_count,
        round(avg(t.tx_fraud), 3) AS fraud_percentage
'''

In [26]:
df_E = read_from_db(query_E, auth)
df_E.head()

The query returned 4 records in 20 ms.


,period_of_day,transaction_count,fraud_percentage
0,night,7,0.857
1,evening,6,0.667
2,afternoon,25,0.840
3,morning,18,0.889


# Transaction Graph (50Mb)

# Transaction Graph (100Mb)

# Transaction Graph (200Mb)